In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os

path_to_project = os.path.abspath(os.path.join(os.getcwd(), '../'))    
sys.path.insert(1, os.path.join(path_to_project))

In [2]:
import re
import pandas as pd
import warnings
from src.data_dict import NHANES_nan_fill
from src.directory import data_dir, NHANES_dir, NHANES_preprocessed_filename, NHANES_vars_lookup_filename
from src.utils import preprocess_NHANES, download_nhanes_xpt

In [3]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# read in variable lookup df
vars_lookup_df = pd.read_csv(os.path.join(data_dir, NHANES_vars_lookup_filename))

# get questionnaire names
questionnaires = vars_lookup_df['Data File Name'].apply(lambda x: re.findall('\(([^)]+)', x)[0]).unique()

url_list = [
    f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/{questionnaire}.xpt" for questionnaire in questionnaires
]

# download datasets (if necessary)
download_nhanes_xpt(url_list)

PAXMIN_H.xpt already exists. Skipping.
SLQ_H.xpt already exists. Skipping.
BPX_H.xpt already exists. Skipping.
BPQ_H.xpt already exists. Skipping.
DIQ_H.xpt already exists. Skipping.
BMX_H.xpt already exists. Skipping.
SMQ_H.xpt already exists. Skipping.
DEMO_H.xpt already exists. Skipping.
DPQ_H.xpt already exists. Skipping.
RXQ_DRUG.xpt already exists. Skipping.
RXQ_RX_H.xpt already exists. Skipping.
PAQ_H.xpt already exists. Skipping.
PAXDAY_H.xpt already exists. Skipping.


In [4]:
NHANES_preprocessed_filepath = os.path.join(data_dir, NHANES_preprocessed_filename)

In [ ]:
df = preprocess_NHANES(exclude=['RXQ_DRUG.xpt', 'PAXMIN_H.xpt'])
df.to_csv(NHANES_preprocessed_filepath)

In [88]:
# get indices for rows with valid HTN or sleep deprivation values
# valid_HTN_or_deprivation = ~(df['HTN'].isna() & df['sleep_deprivation'].isna())
valid_HTN_and_deprivation = ~(df['HTN'].isna() | df['sleep_deprivation'].isna())

total_valid_subjects = valid_HTN_and_deprivation.sum().item()
print('Total subject with valid HTN & sleep deprivation values:', total_valid_subjects)

# get ratio of valid responses for each column
ratio_of_valid_responses = df[valid_HTN_and_deprivation].describe().loc['count'] / total_valid_subjects
ratio_of_valid_responses.sort_values(ascending=False)

Total subject with valid HTN & sleep deprivation values: 6454


physical_activity              1.000000
ANTIDEPRESSANTS_ANXIOLYTICS    1.000000
sleep_troubles                 1.000000
sleep_deprivation              1.000000
diabetes                       1.000000
race_ethnicity                 1.000000
gender                         1.000000
age                            1.000000
HTN                            1.000000
daily_sedentary                0.996281
BMI                            0.957081
poverty_ratio                  0.920205
depression                     0.916641
accelerometer                  0.840564
smoker                         0.399132
Name: count, dtype: float64

In [6]:
# read in lux values
lux_df = preprocess_NHANES(exclude=[
    os.path.basename(x) for x in os.listdir(NHANES_dir) if 'PAXMIN_H.xpt' not in x])

Preprocessing PAXMIN_H.xpt...
1000000.0
2000000.0
3000000.0
4000000.0
5000000.0
6000000.0
7000000.0
8000000.0
9000000.0
10000000.0
11000000.0
12000000.0
13000000.0
14000000.0
15000000.0
16000000.0
17000000.0
18000000.0
19000000.0
20000000.0
21000000.0
22000000.0
23000000.0
24000000.0
25000000.0
26000000.0
27000000.0
28000000.0
29000000.0
30000000.0
31000000.0
32000000.0
33000000.0
34000000.0
35000000.0
36000000.0
37000000.0
38000000.0
39000000.0
40000000.0
41000000.0
42000000.0
43000000.0
44000000.0
45000000.0
46000000.0
47000000.0
48000000.0
49000000.0
50000000.0
51000000.0
52000000.0
53000000.0
54000000.0
55000000.0
56000000.0
57000000.0
58000000.0
59000000.0
60000000.0
61000000.0
62000000.0
63000000.0
64000000.0
65000000.0
66000000.0
67000000.0
68000000.0
69000000.0
70000000.0
71000000.0
72000000.0
73000000.0
74000000.0
75000000.0
76000000.0
77000000.0
78000000.0
79000000.0
80000000.0
81000000.0
82000000.0
83000000.0
84000000.0
85000000.0
86000000.0
87000000.0
88000000.0


In [7]:
lux_df

,ambient_light
SEQN,
73664.0,0.000000
73665.0,836.735051
73666.0,4936.009992
73667.0,0.000000
73668.0,64.910539
...,...
83725.0,0.000000
83727.0,50857.547541
83729.0,1204.640125


In [ ]:
# merge with df
df = pd.concat([df, lux_df], axis=1)
df.to_csv(NHANES_preprocessed_filepath)

,physical_activity,daily_sedentary,depression,accelerometer,ANTIDEPRESSANTS_ANXIOLYTICS,sleep_troubles,sleep_deprivation,diabetes,BMI,smoker,race_ethnicity,gender,age,poverty_ratio,HTN,ambient_light
SEQN,,,,,,,,,,,,,,,,
73557.0,0.0,0.416667,0.0,6369.495333,0,1.0,1.0,1.0,26.7,0.0,4.0,0.0,6.900000e+01,0.84,1,NaN
73558.0,1.0,0.375000,0.0,8307.157778,0,0.0,0.0,1.0,28.6,1.0,3.0,0.0,5.400000e+01,1.78,1,NaN
73559.0,1.0,0.208333,0.0,8005.963333,0,0.0,0.0,1.0,28.9,0.0,3.0,0.0,7.200000e+01,4.51,1,NaN
73560.0,0.0,NaN,NaN,20811.614333,0,NaN,NaN,0.0,17.1,NaN,3.0,0.0,9.000000e+00,2.52,0,NaN
73561.0,1.0,0.333333,1.0,6470.660889,0,0.0,0.0,0.0,19.7,NaN,3.0,1.0,7.300000e+01,5.00,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83661.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,3.0,0.0,1.000000e+00,1.07,0,NaN
83681.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,6.0,1.0,1.000000e+00,1.02,0,NaN
83685.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,5.397605e-79,2.82,0,NaN


In [ ]:
# TODO: fill nan/impute missing values
for col, fill_value in NHANES_nan_fill.items():
    if col in df.columns:
        df[col] = df[col].fillna(fill_value)